In [1]:
import os
import argparse
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D



Using TensorFlow backend.


In [9]:
def get_commaai_model(img_shape):
    """
    Creates the comma.ai model, and returns a reference to the model
    The comma.ai model's original source code is available at:
    https://github.com/commaai/research/blob/master/train_steering_model.py
    """
    row, col, ch = img_shape  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(row, col, ch),
        output_shape=(row, col, ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return model

In [3]:
import numpy as np
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt
import pickle

log_file = 'sim_data/driving_log.csv'
df = pd.read_csv(log_file, sep=',',header=None)

print(df.head())

                                                   0   1   2    3    4    5  \
0  ./sim_data/IMG/center_2017_02_06_19_43_06_277.jpg NaN NaN  0.0  0.0  0.0   
1  ./sim_data/IMG/center_2017_02_06_19_43_06_382.jpg NaN NaN  0.0  0.0  0.0   
2  ./sim_data/IMG/center_2017_02_06_19_43_06_498.jpg NaN NaN  0.0  0.0  0.0   
3  ./sim_data/IMG/center_2017_02_06_19_43_06_614.jpg NaN NaN  0.0  0.0  0.0   
4  ./sim_data/IMG/center_2017_02_06_19_43_06_719.jpg NaN NaN  0.0  0.0  0.0   

          6  
0  0.000007  
1  0.000011  
2  0.000009  
3  0.000013  
4  0.000007  


In [4]:

img_shape = imread(df[0][0]).shape
features = np.zeros([len(df), img_shape[0], img_shape[1], img_shape[2]])
labels = np.zeros([len(df), 1])

for i, row in df.iterrows():
    features[i] = imread(row[0])
    labels[i] = row[3]

# data = {
#     'features': features,
#     'labels': labels
# }

# with open('train.p', 'wr') as f:
#     pickle.dump(data, f)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3)

In [11]:
print(X_train.shape[1:])

model = get_commaai_model(X_train.shape[1:])

history = model.fit(X_train, y_train)
model.save('model.h5')

(160, 320, 3)
Epoch 1/10
1901/1901 [==============================] - 4s - loss: 0.1955 - acc: 0.0195     
Epoch 2/10
1901/1901 [==============================] - 4s - loss: 0.0581 - acc: 0.0216     
Epoch 3/10
1901/1901 [==============================] - 4s - loss: 0.0369 - acc: 0.0268     
Epoch 4/10
1901/1901 [==============================] - 4s - loss: 0.0321 - acc: 0.0289     
Epoch 5/10
1901/1901 [==============================] - 4s - loss: 0.0269 - acc: 0.0300     
Epoch 6/10
1901/1901 [==============================] - 4s - loss: 0.0291 - acc: 0.0289     
Epoch 7/10
1901/1901 [==============================] - 4s - loss: 0.0281 - acc: 0.0305     
Epoch 8/10
1901/1901 [==============================] - 4s - loss: 0.0282 - acc: 0.0305     
Epoch 9/10
1901/1901 [==============================] - 4s - loss: 0.0250 - acc: 0.0321     
Epoch 10/10
1901/1901 [==============================] - 4s - loss: 0.0225 - acc: 0.0305     


In [13]:
print(history.history)

{'loss': [0.1954845493793613, 0.058064673801414717, 0.03693534135975253, 0.032086640935408826, 0.026911446915793457, 0.02914836869498167, 0.028137461203437051, 0.028235089660929354, 0.025016887357428099, 0.022514220339141744], 'acc': [0.019463440294581798, 0.021567596041297109, 0.026827985270910047, 0.02893214099802888, 0.029984218832193581, 0.028932140978432403, 0.030510257778670648, 0.030510257759074171, 0.032088374559312419, 0.030510257759074171]}


In [14]:
results = model.evaluate(X_val, y_val)
print(results)

815/815 [==============================] - 1s     
[0.062781790794770406, 0.019631901840490799]
